In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from arctic_objects import *
from arctic_functions import *
import scipy.io as sio
import netCDF4 as nc
import numpy as np

In [3]:
# Get paths to mitGCM data files
mitGCM_filepath = '/ocean/jolerner/data/mitGCM/'
mitGCM_ncfiles = ['Arctic_CTLnew.nc', 
                'Arctic_LOnew.nc', 
                'Arctic_REGnew.nc', 
                'Arctic_HInew.nc', 
                'Arctic_OBSnew.nc']
NEMO_filepath = '/ocean/jolerner/data/NEMO/'
NEMO_ncfiles = ['ANHA4-EXH005_2012-2016avg_votkeavt.nc', 
                'ANHA4-EXH005_2012-2016avg_vosaline.nc',
                'ANHA4-EXH005_2012-2016avg_votemper.nc']

In [4]:
# Build the necessary grid data for mitGCM fields
mitGCM_grid = {'lat': None, 'lon': None, 'Depth': None, 'vol': None, 'ocnmsk': None}
for key in mitGCM_grid.keys():
    mitGCM_grid[key] = sio.loadmat(mitGCM_filepath + 'mgrid')['mgrid'][key][0,0]
mitGCM_grid['Depth'] = np.squeeze(mitGCM_grid['Depth'])

In [5]:
#Build the necessary regional logicals for mitGCM fields
mitGCM_reg = {'ARC': None, 'CB': None, 'EB': None, 'NAt': None, 'shelf': None, 'slope': None}
for key in mitGCM_reg.keys():
    mitGCM_reg[key] = sio.loadmat(mitGCM_filepath + 'regional_logicals')['ii_' + key]

In [6]:
# Build the necessary grid data for NEMO fields
NEMO_grid = {'lat': None, 'lon': None, 'Depth': None, 'vol': None, 'ocnmsk': None}
mesh = nc.Dataset(NEMO_filepath + 'mesh_mask_anha4_201509.nc')
NEMO_grid['lat'] = np.array(mesh.variables['nav_lat'])
NEMO_grid['lon'] = np.array(mesh.variables['nav_lon'])
NEMO_grid['Depth'] = -1*np.array(mesh.variables['nav_lev'])
NEMO_grid['vol'] = np.moveaxis(mesh.variables['e1t'][0]*mesh.variables['e2t'][0]*mesh.variables['e3t'][0], 0, -1)
NEMO_grid['ocnmsk'] = np.moveaxis(mesh.variables['tmask'][0], 0, -1)
NEMO_reg = makeRegions(NEMO_grid)

In [7]:
# Make Transects
canbar_start = np.array([-144.3258153, 70.29163412])
canbar_end = np.array([25.05097782, 71.335544347])

fram_start = np.array([-16.626389, 80.676944])
fram_end = np.array([11.748611, 79.158611])

mitGCM_canbarTransect = Transect('CANBAR Transect', mitGCM_grid, canbar_start, canbar_end)
mitGCM_framTransect = Transect('FRAM Transect', mitGCM_grid, fram_start, fram_end)
NEMO_canbarTransect = Transect('CANBAR Transect', NEMO_grid, canbar_start, canbar_end)
NEMO_framTransect = Transect('FRAM Transect', NEMO_grid, fram_start, fram_end)

In [8]:
# Make mitGCM kappa background fields
kappa_obs = sio.loadmat(mitGCM_filepath + 'kappa_OBS')['kappa']
mitGCM_OBS_kappaBG = KappaBG('mitGCM Observational', mitGCM_grid, kappa_obs, reg_logic=mitGCM_reg)

In [9]:
# Load mitGCM vertical diffusivity and diffusive flux fields
mitGCM_CTLvars = nc.Dataset(mitGCM_filepath + 'nc_files/' + mitGCM_ncfiles[0]).variables
mitGCM_OBSvars = nc.Dataset(mitGCM_filepath + 'nc_files/' + mitGCM_ncfiles[-1]).variables

# Control
mitGCM_CTL_kappaT = Field('mitGCM Control', 'Vertical Eddy Diffusivity', r'$m^2/s$', mitGCM_grid, 
                          yearAvg(mitGCM_CTLvars, 'kappaT', (2008, 2013)), reg_logic=mitGCM_reg)
mitGCM_CTL_IdiffT = Field('mitGCM Control (Implicit)', 'Vertical Diffusive Heat Flux', r'$°C~m^3/s$', mitGCM_grid, 
                          yearAvg(mitGCM_CTLvars, 'DIFFz_TH_I', (2008, 2013)), reg_logic=mitGCM_reg)
mitGCM_CTL_EdiffT = Field('mitGCM Control (Explicit)', 'Vertical Diffusive Heat Flux', r'$°C~m^3/s$', mitGCM_grid, 
                          yearAvg(mitGCM_CTLvars, 'DIFFz_TH_E', (2008, 2013)), reg_logic=mitGCM_reg)
mitGCM_CTL_IdiffS = Field('mitGCM Control (Implicit)', 'Vertical Diffusive Salt Flux', r'$PSU~m^3/s$', mitGCM_grid, 
                          yearAvg(mitGCM_CTLvars, 'DIFFz_SLT_I', (2008, 2013)), reg_logic=mitGCM_reg)
mitGCM_CTL_EdiffS = Field('mitGCM Control (Explicit)', 'Vertical Diffusive Salt Flux', r'$PSU~m^3/s$', mitGCM_grid, 
                          yearAvg(mitGCM_CTLvars, 'DIFFz_SLT_E', (2008, 2013)), reg_logic=mitGCM_reg)
mitGCM_CTL_temp = Field('mitGCM Control', 'Temperature', '°C', mitGCM_grid, 
                          yearAvg(mitGCM_CTLvars, 'theta', (2008, 2013)), reg_logic=mitGCM_reg)
mitGCM_CTL_salt = Field('mitGCM Control', 'Salinity', 'PSU', mitGCM_grid, 
                          yearAvg(mitGCM_CTLvars, 'salinity', (2008, 2013)), reg_logic=mitGCM_reg)
# Observational
mitGCM_OBS_kappaT = Field('mitGCM Observational', 'Vertical Eddy Diffusivity', r'$m^2/s$', mitGCM_grid, 
                          yearAvg(mitGCM_OBSvars, 'kappaT', (2008, 2013)), reg_logic=mitGCM_reg)
mitGCM_OBS_IdiffT = Field('mitGCM Observational (Implicit)', 'Vertical Diffusive Heat Flux', r'$°C~m^3/s$', mitGCM_grid, 
                          yearAvg(mitGCM_OBSvars, 'DIFFz_TH_I', (2008, 2013)), reg_logic=mitGCM_reg)
mitGCM_OBS_EdiffT = Field('mitGCM Observational (Explicit)', 'Vertical Diffusive Heat Flux', r'$°C~m^3/s$', mitGCM_grid, 
                          yearAvg(mitGCM_OBSvars, 'DIFFz_TH_E', (2008, 2013)), reg_logic=mitGCM_reg)
mitGCM_OBS_IdiffS = Field('mitGCM Observational (Implicit)', 'Vertical Diffusive Salt Flux', r'$PSU~m^3/s$', mitGCM_grid, 
                          yearAvg(mitGCM_OBSvars, 'DIFFz_SLT_I', (2008, 2013)), reg_logic=mitGCM_reg)
mitGCM_OBS_EdiffS = Field('mitGCM Observational (Explicit)', 'Vertical Diffusive Salt Flux', r'$PSU~m^3/s$', mitGCM_grid, 
                          yearAvg(mitGCM_OBSvars, 'DIFFz_SLT_E', (2008, 2013)), reg_logic=mitGCM_reg)
mitGCM_OBS_temp = Field('mitGCM Observational', 'Temperature', '°C', mitGCM_grid, 
                          yearAvg(mitGCM_OBSvars, 'theta', (2008, 2013)), reg_logic=mitGCM_reg)
mitGCM_OBS_salt = Field('mitGCM Observational', 'Salinity', 'PSU', mitGCM_grid, 
                          yearAvg(mitGCM_OBSvars, 'salinity', (2008, 2013)), reg_logic=mitGCM_reg)

# Comparison
mitGCM_CMP_kappaT = Field('mitGCM Observational/Control', 'Vertical Eddy Diffusivity Fraction', '1', mitGCM_grid, 
                          mitGCM_OBS_kappaT.getData()/zero2Nan(mitGCM_CTL_kappaT.getData()), reg_logic=mitGCM_reg)
mitGCM_CMP_IdiffT = Field('mitGCM Observational - Control (Implicit)', 'Vertical Diffusive Heat Flux', r'$°C~m^3/s$', mitGCM_grid, 
                          mitGCM_OBS_IdiffT.getData() - mitGCM_CTL_IdiffT.getData(), reg_logic=mitGCM_reg)
mitGCM_CMP_EdiffT = Field('mitGCM Observational - Control (Explicit)', 'Vertical Diffusive Heat Flux', r'$°C~m^3/s$', mitGCM_grid, 
                          mitGCM_OBS_EdiffT.getData() - mitGCM_CTL_EdiffT.getData(), reg_logic=mitGCM_reg)
mitGCM_CMP_IdiffS = Field('mitGCM Observational - Control (Implicit)', 'Vertical Diffusive Salt Flux', r'$PSU~m^3/s$', mitGCM_grid, 
                          mitGCM_OBS_IdiffS.getData() - mitGCM_CTL_IdiffS.getData(), reg_logic=mitGCM_reg)
mitGCM_CMP_EdiffS = Field('mitGCM Observational - Control (Explicit)', 'Vertical Diffusive Salt Flux', r'$PSU~m^3/s$', mitGCM_grid, 
                          mitGCM_OBS_EdiffS.getData() - mitGCM_CTL_EdiffS.getData(), reg_logic=mitGCM_reg)
mitGCM_CMP_temp = Field('mitGCM Observational - Control', 'Temperature', '°C', mitGCM_grid, 
                          mitGCM_OBS_temp.getData() - mitGCM_CTL_temp.getData(), reg_logic=mitGCM_reg)
mitGCM_CMP_salt = Field('mitGCM Observational - Control', 'Salinity', 'PSU', mitGCM_grid, 
                          mitGCM_OBS_salt.getData() - mitGCM_CTL_salt.getData(), reg_logic=mitGCM_reg)


In [10]:
# Make NEMO temperature, salinity, and vertical eddy diffusivity field

NEMO_CTL_votkeavt = np.moveaxis(nc.Dataset(NEMO_filepath + NEMO_ncfiles[0]).variables['votkeavtfour'], 0, -1)
NEMO_CTL_vosaline = np.moveaxis(nc.Dataset(NEMO_filepath + NEMO_ncfiles[1]).variables['vosalinefour'], 0, -1)
NEMO_CTL_votemper = np.moveaxis(nc.Dataset(NEMO_filepath + NEMO_ncfiles[2]).variables['votemperfour'], 0, -1)

NEMO_CTL_kappaFull = Field('NEMO Control', 'Vertical Eddy Diffusivity', r'$m^2/s$',  NEMO_grid, NEMO_CTL_votkeavt, reg_logic=NEMO_reg)
NEMO_CTL_salt = Field('NEMO Control', 'Salinity', 'PSU',  NEMO_grid, NEMO_CTL_vosaline, reg_logic=NEMO_reg)
NEMO_CTL_temp = Field('NEMO Control', 'Temperature', '°C',  NEMO_grid, NEMO_CTL_votemper, reg_logic=NEMO_reg)

In [11]:
# Visualize mitGCM vertical diffusivity and diffusive flux fields
mitGCM_savepath = '/ocean/jolerner/plots/mitGCM/'

# Control
mitGCM_CTL_kappaT.visualize_regional_profile(scale='log').savefig(mitGCM_savepath+'Control/CTL_kappaT_AverageProfile.png')
mitGCM_CTL_kappaT.visualize_distributions(scale='log').savefig(mitGCM_savepath+'Control/CTL_kappaT_Distributions.png')
mitGCM_CTL_kappaT.visualize_maps(scale='log').savefig(mitGCM_savepath+'Control/CTL_kappaT_Maps.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_CTL_kappaT, scale='log').savefig(mitGCM_savepath+'Control/CTL_kappaT_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_CTL_kappaT, scale='log').savefig(mitGCM_savepath+'Control/CTL_kappaT_FramTransect.png')

mitGCM_CTL_IdiffT.visualize_regional_profile(scale='symlog').savefig(mitGCM_savepath+'Control/CTL_diffzT_Implicit_AverageProfile.png')
mitGCM_CTL_IdiffT.visualize_distributions(scale='symlog').savefig(mitGCM_savepath+'Control/CTL_diffzT_Implicit_Distributions.png')
mitGCM_CTL_IdiffT.visualize_maps(scale='symlog').savefig(mitGCM_savepath+'Control/CTL_diffzT_Implicit_Maps.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_CTL_IdiffT, scale='symlog').savefig(mitGCM_savepath+'Control/CTL_diffzT_Implicit_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_CTL_IdiffT, scale='symlog').savefig(mitGCM_savepath+'Control/CTL_diffzT_Implicit_FramTransect.png')

mitGCM_CTL_EdiffT.visualize_regional_profile(scale='symlog').savefig(mitGCM_savepath+'Control/CTL_diffzT_Explicit_AverageProfile.png')
mitGCM_CTL_EdiffT.visualize_distributions(scale='symlog').savefig(mitGCM_savepath+'Control/CTL_diffzT_Explicit_Distributions.png')
mitGCM_CTL_EdiffT.visualize_maps(scale='symlog').savefig(mitGCM_savepath+'Control/CTL_diffzT_Explicit_Maps.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_CTL_EdiffT, scale='symlog').savefig(mitGCM_savepath+'Control/CTL_diffzT_Explicit_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_CTL_EdiffT, scale='symlog').savefig(mitGCM_savepath+'Control/CTL_diffzT_Explicit_FramTransect.png')

mitGCM_CTL_IdiffS.visualize_regional_profile(scale='symlog').savefig(mitGCM_savepath+'Control/CTL_diffzS_Implicit_AverageProfile.png')
mitGCM_CTL_IdiffS.visualize_distributions(scale='symlog').savefig(mitGCM_savepath+'Control/CTL_diffzS_Implicit_Distributions.png')
mitGCM_CTL_IdiffS.visualize_maps(scale='symlog').savefig(mitGCM_savepath+'Control/CTL_diffzS_Implicit_Maps.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_CTL_IdiffS, scale='symlog').savefig(mitGCM_savepath+'Control/CTL_diffzS_Implicit_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_CTL_IdiffS, scale='symlog').savefig(mitGCM_savepath+'Control/CTL_diffzS_Implicit_FramTransect.png')

mitGCM_CTL_EdiffS.visualize_regional_profile(scale='symlog').savefig(mitGCM_savepath+'Control/CTL_diffzS_Explicit_AverageProfile.png')
mitGCM_CTL_EdiffS.visualize_distributions(scale='symlog').savefig(mitGCM_savepath+'Control/CTL_diffzS_Explicit_Distributions.png')
mitGCM_CTL_EdiffS.visualize_maps(scale='symlog').savefig(mitGCM_savepath+'Control/CTL_diffzS_Explicit_Maps.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_CTL_EdiffS, scale='symlog').savefig(mitGCM_savepath+'Control/CTL_diffzS_Explicit_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_CTL_EdiffS, scale='symlog').savefig(mitGCM_savepath+'Control/CTL_diffzS_Explicit_FramTransect.png')

mitGCM_CTL_temp.visualize_regional_profile().savefig(mitGCM_savepath+'Control/CTL_temp_AverageProfile.png')
mitGCM_CTL_temp.visualize_distributions().savefig(mitGCM_savepath+'Control/CTL_temp_Distributions.png')
mitGCM_CTL_temp.visualize_maps().savefig(mitGCM_savepath+'Control/CTL_temp_Maps.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_CTL_temp).savefig(mitGCM_savepath+'Control/CTL_temp_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_CTL_temp).savefig(mitGCM_savepath+'Control/CTL_temp_FramTransect.png')

mitGCM_CTL_salt.visualize_regional_profile().savefig(mitGCM_savepath+'Control/CTL_temp_AverageProfile.png')
mitGCM_CTL_salt.visualize_distributions().savefig(mitGCM_savepath+'Control/CTL_temp_Distributions.png')
mitGCM_CTL_salt.visualize_maps().savefig(mitGCM_savepath+'Control/CTL_temp_Maps.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_CTL_salt).savefig(mitGCM_savepath+'Control/CTL_salt_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_CTL_salt).savefig(mitGCM_savepath+'Control/CTL_salt_FramTransect.png')

volumeCensus(mitGCM_CTL_salt, mitGCM_CTL_temp).savefig(mitGCM_savepath+'Control/CTL_temp_salt_volumeCensus.png')
volumeCensus(mitGCM_CTL_salt, mitGCM_CTL_temp, range=((22.5, 40),(-4, 30))).savefig(mitGCM_savepath+'Control/cropped_CTL_temp_salt_volumeCensus.png')

In [12]:
# Observational
mitGCM_OBS_kappaBG.visualize_regional_profile(scale='log').savefig(mitGCM_savepath+'Observational/OBS_kappaBG_AverageProfile.png')
mitGCM_OBS_kappaBG.visualize_distributions(scale='log').savefig(mitGCM_savepath+'Observational/OBS_kappaBG_Distributions.png')
mitGCM_OBS_kappaBG.visualize_maps(scale='log').savefig(mitGCM_savepath+'Observational/OBS_kappaBG_Maps.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_OBS_kappaBG, scale='log').savefig(mitGCM_savepath+'Observational/OBS_kappaBG_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_OBS_kappaBG, scale='log').savefig(mitGCM_savepath+'Observational/OBS_kappaBG_FramTransect.png')

mitGCM_OBS_kappaT.visualize_regional_profile(scale='log').savefig(mitGCM_savepath+'Observational/OBS_kappaT_AverageProfile.png')
mitGCM_OBS_kappaT.visualize_distributions(scale='log').savefig(mitGCM_savepath+'Observational/OBS_kappaT_Distributions.png')
mitGCM_OBS_kappaT.visualize_maps(scale='log').savefig(mitGCM_savepath+'Observational/OBS_kappaT_Maps.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_OBS_kappaT, scale='log').savefig(mitGCM_savepath+'Observational/OBS_kappaT_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_OBS_kappaT, scale='log').savefig(mitGCM_savepath+'Observational/OBS_kappaT_FramTransect.png')

mitGCM_OBS_IdiffT.visualize_regional_profile(scale='symlog').savefig(mitGCM_savepath+'Observational/OBS_diffzT_Implicit_AverageProfile.png')
mitGCM_OBS_IdiffT.visualize_distributions(scale='symlog').savefig(mitGCM_savepath+'Observational/OBS_diffzT_Implicit_Distributions.png')
mitGCM_OBS_IdiffT.visualize_maps(scale='symlog').savefig(mitGCM_savepath+'Observational/OBS_diffzT_Implicit_Maps.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_OBS_IdiffT, scale='symlog').savefig(mitGCM_savepath+'Observational/OBS_diffzT_Implicit_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_OBS_IdiffT, scale='symlog').savefig(mitGCM_savepath+'Observational/OBS_diffzT_Implicit_FramTransect.png')

mitGCM_OBS_EdiffT.visualize_regional_profile(scale='symlog').savefig(mitGCM_savepath+'Observational/OBS_diffzT_Explicit_AverageProfile.png')
mitGCM_OBS_EdiffT.visualize_distributions(scale='symlog').savefig(mitGCM_savepath+'Observational/OBS_diffzT_Explicit_Distributions.png')
mitGCM_OBS_EdiffT.visualize_maps(scale='symlog').savefig(mitGCM_savepath+'Observational/OBS_diffzT_Explicit_Maps.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_OBS_EdiffT, scale='symlog').savefig(mitGCM_savepath+'Observational/OBS_diffzT_Explicit_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_OBS_EdiffT, scale='symlog').savefig(mitGCM_savepath+'Observational/OBS_diffzT_Explicit_FramTransect.png')

mitGCM_OBS_IdiffS.visualize_regional_profile(scale='symlog').savefig(mitGCM_savepath+'Observational/OBS_diffzS_Implicit_AverageProfile.png')
mitGCM_OBS_IdiffS.visualize_distributions(scale='symlog').savefig(mitGCM_savepath+'Observational/OBS_diffzS_Implicit_Distributions.png')
mitGCM_OBS_IdiffS.visualize_maps(scale='symlog').savefig(mitGCM_savepath+'Observational/OBS_diffzS_Implicit_Maps.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_OBS_IdiffS, scale='symlog').savefig(mitGCM_savepath+'Observational/OBS_diffzS_Implicit_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_OBS_IdiffS, scale='symlog').savefig(mitGCM_savepath+'Observational/OBS_diffzS_Implicit_FramTransect.png')

mitGCM_OBS_EdiffS.visualize_regional_profile(scale='symlog').savefig(mitGCM_savepath+'Observational/OBS_diffzS_Explicit_AverageProfile.png')
mitGCM_OBS_EdiffS.visualize_distributions(scale='symlog').savefig(mitGCM_savepath+'Observational/OBS_diffzS_Explicit_Distributions.png')
mitGCM_OBS_EdiffS.visualize_maps(scale='symlog').savefig(mitGCM_savepath+'Observational/OBS_diffzS_Explicit_Maps.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_OBS_EdiffS, scale='symlog').savefig(mitGCM_savepath+'Observational/OBS_diffzS_Explicit_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_OBS_EdiffS, scale='symlog').savefig(mitGCM_savepath+'Observational/OBS_diffzS_Explicit_FramTransect.png')

mitGCM_OBS_temp.visualize_regional_profile().savefig(mitGCM_savepath+'Observational/OBS_temp_AverageProfile.png')
mitGCM_OBS_temp.visualize_distributions().savefig(mitGCM_savepath+'Observational/OBS_temp_Distributions.png')
mitGCM_OBS_temp.visualize_maps().savefig(mitGCM_savepath+'Observational/OBS_temp_Maps.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_OBS_temp).savefig(mitGCM_savepath+'Observational/OBS_temp_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_OBS_temp).savefig(mitGCM_savepath+'Observational/OBS_temp_FramTransect.png')

mitGCM_OBS_salt.visualize_regional_profile().savefig(mitGCM_savepath+'Observational/OBS_temp_AverageProfile.png')
mitGCM_OBS_salt.visualize_distributions().savefig(mitGCM_savepath+'Observational/OBS_temp_Distributions.png')
mitGCM_OBS_salt.visualize_maps().savefig(mitGCM_savepath+'Observational/OBS_temp_Maps.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_OBS_salt).savefig(mitGCM_savepath+'Observational/OBS_salt_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_OBS_salt).savefig(mitGCM_savepath+'Observational/OBS_salt_FramTransect.png')

volumeCensus(mitGCM_OBS_salt, mitGCM_OBS_temp).savefig(mitGCM_savepath+'Observational/OBS_temp_salt_volumeCensus.png')
volumeCensus(mitGCM_OBS_salt, mitGCM_OBS_temp, range=((22.5, 40),(-4, 30))).savefig(mitGCM_savepath+'Observational/cropped_OBS_temp_salt_volumeCensus.png')

In [14]:
# Comparison
visualize_average_profiles([mitGCM_CTL_kappaT, mitGCM_OBS_kappaT], scale='log').savefig(mitGCM_savepath+'Comparison/kappaT_CompareProfile.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_CMP_kappaT).savefig(mitGCM_savepath+'Comparison/CMP_kappaT_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_CMP_kappaT).savefig(mitGCM_savepath+'Comparison/CMP_kappaT_FramTransect.png')

visualize_average_profiles([mitGCM_CTL_IdiffT, mitGCM_OBS_IdiffT], scale='log').savefig(mitGCM_savepath+'Comparison/diffzT_Implicit_CompareProfile.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_CMP_IdiffT).savefig(mitGCM_savepath+'Comparison/CMP_diffzT_Implicit_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_CMP_IdiffT).savefig(mitGCM_savepath+'Comparison/CMP_diffzT_Implicit_FramTransect.png')

visualize_average_profiles([mitGCM_CTL_EdiffT, mitGCM_OBS_EdiffT], scale='log').savefig(mitGCM_savepath+'Comparison/diffzT_Explicit_CompareProfile.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_CMP_EdiffT).savefig(mitGCM_savepath+'Comparison/CMP_diffzT_Explicit_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_CMP_EdiffT).savefig(mitGCM_savepath+'Comparison/CMP_diffzT_Explicit_FramTransect.png')

visualize_average_profiles([mitGCM_CTL_IdiffS, mitGCM_OBS_IdiffS], scale='log').savefig(mitGCM_savepath+'Comparison/diffzS_Implicit_CompareProfile.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_CMP_IdiffS).savefig(mitGCM_savepath+'Comparison/CMP_diffzS_Implicit_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_CMP_IdiffS).savefig(mitGCM_savepath+'Comparison/CMP_diffzS_Implicit_FramTransect.png')

visualize_average_profiles([mitGCM_CTL_EdiffS, mitGCM_OBS_EdiffS], scale='log').savefig(mitGCM_savepath+'Comparison/diffzS_Explicit_CompareProfile.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_CMP_EdiffS).savefig(mitGCM_savepath+'Comparison/CMP_diffzS_Explicit_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_CMP_EdiffS).savefig(mitGCM_savepath+'Comparison/CMP_diffzS_Explicit_FramTransect.png')

visualize_average_profiles([mitGCM_CTL_temp, mitGCM_OBS_temp]).savefig(mitGCM_savepath+'Comparison/temp_CompareProfile.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_CMP_temp).savefig(mitGCM_savepath+'Comparison/CMP_temp_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_CMP_temp).savefig(mitGCM_savepath+'Comparison/CMP_temp_FramTransect.png')

visualize_average_profiles([mitGCM_CTL_salt, mitGCM_OBS_salt]).savefig(mitGCM_savepath+'Comparison/salt_CompareProfile.png')
mitGCM_canbarTransect.visualize_transect(mitGCM_CMP_salt).savefig(mitGCM_savepath+'Comparison/CMP_salt_CanbarTransect.png')
mitGCM_framTransect.visualize_transect(mitGCM_CMP_salt).savefig(mitGCM_savepath+'Comparison/CMP_salt_FramTransect.png')

anomalyVolumeCensus([mitGCM_OBS_salt, mitGCM_CTL_salt], [mitGCM_OBS_temp, mitGCM_CTL_temp], range=((20, 40), (-4, 34))).savefig(mitGCM_savepath+'Comparison/CMP_temp_salt_volumeCensus.png')

mitGCM_canbarTransect.visualize_line().savefig(mitGCM_savepath+'TransectMap_canbar.png')
mitGCM_framTransect.visualize_line().savefig(mitGCM_savepath+'TransectMap_fram.png')

In [15]:
# Visualize NEMO vertical diffusivity
NEMO_savepath = '/ocean/jolerner/plots/NEMO/'

NEMO_CTL_kappaFull.visualize_regional_profile(scale='log').savefig(NEMO_savepath+'Control/CTL_kappaFull_AverageProfile.png')
NEMO_CTL_kappaFull.visualize_distributions(scale='log').savefig(NEMO_savepath+'Control/CTL_kappaFull_Distributions.png')
NEMO_CTL_kappaFull.visualize_maps(scale='log').savefig(NEMO_savepath+'Control/CTL_kappaFull_Maps.png')
NEMO_canbarTransect.visualize_transect(NEMO_CTL_kappaFull, scale='log').savefig(NEMO_savepath+'Control/CTL_kappaFull_CanbarTransect.png')
NEMO_framTransect.visualize_transect(NEMO_CTL_kappaFull, scale='log').savefig(NEMO_savepath+'Control/CTL_kappaFull_FramTransect.png')

NEMO_CTL_temp.visualize_regional_profile().savefig(NEMO_savepath+'Control/CTL_temp_AverageProfile.png')
NEMO_CTL_temp.visualize_distributions().savefig(NEMO_savepath+'Control/CTL_temp_Distributions.png')
NEMO_CTL_temp.visualize_maps().savefig(NEMO_savepath+'Control/CTL_temp_Maps.png')
NEMO_canbarTransect.visualize_transect(NEMO_CTL_temp).savefig(NEMO_savepath+'Control/CTL_temp_CanbarTransect.png')
NEMO_framTransect.visualize_transect(NEMO_CTL_temp).savefig(NEMO_savepath+'Control/CTL_temp_FramTransect.png')

NEMO_CTL_salt.visualize_regional_profile().savefig(NEMO_savepath+'Control/CTL_salt_AverageProfile.png')
NEMO_CTL_salt.visualize_distributions().savefig(NEMO_savepath+'Control/CTL_salt_Distributions.png')
NEMO_CTL_salt.visualize_maps().savefig(NEMO_savepath+'Control/CTL_salt_Maps.png')
NEMO_canbarTransect.visualize_transect(NEMO_CTL_salt).savefig(NEMO_savepath+'Control/CTL_salt_CanbarTransect.png')
NEMO_framTransect.visualize_transect(NEMO_CTL_salt).savefig(NEMO_savepath+'Control/CTL_salt_FramTransect.png')

volumeCensus(NEMO_CTL_salt, NEMO_CTL_temp).savefig(NEMO_savepath+'Control/CTL_temp_salt_volumeCensus.png')
volumeCensus(NEMO_CTL_salt, NEMO_CTL_temp, range=((22.5, 40),(-4, 30))).savefig(NEMO_savepath+'Control/cropped_CTL_temp_salt_volumeCensus.png')

NEMO_canbarTransect.visualize_line().savefig(NEMO_savepath+'TransectMap_canbar.png')
NEMO_framTransect.visualize_line().savefig(NEMO_savepath+'TransectMap_fram.png')

In [13]:
comp_savepath = '/ocean/jolerner/plots/Comparison/'

#visualize_average_profiles([mitGCM_CTL_kappaT, mitGCM_OBS_kappaT, NEMO_CTL_kappaFull], weighted=True, scale='log').savefig(comp_savepath+'kappa_CompareProfile.png')
anomalyVolumeCensus([NEMO_CTL_salt, mitGCM_CTL_salt], [NEMO_CTL_temp, mitGCM_CTL_temp], range=((20, 40), (-4, 34))).savefig(comp_savepath+'temp_salt_anomalyVolumeCensus.png')